In [1]:
from vUtil.vTime import getNow
from vMysql import MysqlProxy

In [2]:
from tmpIncTable.tmpPatents import TmpPatents
from tmpIncTable.tmpProjects import TmpProjects
from tmpIncTable.tmpPapers import TmpPapers
from tmpIncTable.tmpEnterprises import TmpEnterprises
from tmpIncTable.tmpScholars import TmpScholars
from tmpIncTable.tmpScholarsAbroad import TmpScholarsAbroad
from tmpIncTable.tmpProducts import TmpProducts
from tmpIncTable.tmpSoftwareCopyrights import TmpSoftwareCopyrights

In [2]:
import sys
sys.path += ['..']

In [3]:
from timeManager import TimeManagerMysql

In [8]:
tm = TimeManagerMysql('sync_record', 'sync_time', 'es_flag')
tm

< TimeManagerMysql {'last': Timestamp('2021-06-07 16:00:00'), 'now': Timestamp('2021-06-10 16:00:00')} >

In [7]:
print(tm.db.history)


            update `sync_record`
            set `es_flag` = 1
            where not `es_flag`
            and `sync_time` <= Timestamp('2021-06-10 16:00:00')
        


In [6]:
tm.close()

In [15]:
flagCol = 'es_flag'
timeCol = 'sync_time'
table = 'sync_record'
a = tm.db.select(timeCol, table, 1, where=flagCol, order=f'by {timeCol} desc')

In [22]:
a.

('sync_time',
 0   2021-06-07 16:00:00
 Name: sync_time, dtype: datetime64[ns])

In [3]:
db = MysqlProxy()

In [11]:
order\ by = 1

SyntaxError: unexpected character after line continuation character (<ipython-input-11-16c82fe4237d>, line 1)

In [9]:
db.select('*', 'landinn_restore.sync_record', 1, where='es_flag', order ='by id desc')['sync_time'].item()

Timestamp('2021-06-07 16:00:00')

In [7]:
db.select('*', '', 1, where='es_flag order by id desc')['sync_time'].item()
db.sql('''
select * from landinn_restore.sync_record where es_flag order by id desc 
''')

Timestamp('2021-06-07 16:00:00')

In [8]:
db.select('*', 'landinn_restore.sync_record', 1, where='not es_flag order by id desc')['sync_time'].item()

Timestamp('2021-06-10 14:00:00')

In [9]:
tLast = '2021-06-01'
tNow = getNow()

In [10]:
tmp = TmpPatents(db)

In [11]:
tmp.start(tLast, tNow)

In [12]:
len(tmp)

31131

In [13]:
tmp.close()

In [13]:
from tmpIncTable.tmpTemplet import getSqlInsert
from utils import stdSqlData

In [5]:
tableTmp = 'temp_db.tmp_patents'
print(getSqlInsert(tableTmp, 'patent', 'golaxy_patent_id', containDel=0),)


    replace into temp_db.tmp_patents
    (
    	select  golaxy_patent_id as id from patent
        where %s <= if (updated_at is null, created_at, updated_at)
            and if (updated_at is null, created_at, updated_at) < %s
            and !ifnull(is_deleted, 0)
    );
    


In [6]:
sCreate = '''
create table if not exists %s
(id bigint not null, primary key (id));
'''
sRmTmp = '''
drop table if exists %s;
'''

In [7]:
db.sql(sRmTmp % tableTmp, 1)

""


In [8]:
db.sql(sCreate % tableTmp, 1)

""


In [8]:
tLast = '2021-06-03'
tNow = getNow()

In [15]:
db.sql(getSqlInsert(tableTmp, 'patent', 'golaxy_patent_id', containDel=0) % (tLast, tNow))

""


In [20]:
db.close()

In [9]:
tmp = TmpPatents(db)

In [10]:
tmp.start(tLast, tNow)

In [11]:
len(tmp)

1016

In [12]:
tmp.close()

In [20]:
where = '1'
where = [
        'ifnull(is_deleted,0)',
        (f'{stdSqlData(tLast)} <= if (updated_at is null, created_at, updated_at)' if tLast else ''),
        (f'if (updated_at is null, created_at, updated_at) < {stdSqlData(tNow)}' if tNow else ''),
        (f'({where})' if where else ''),
    ]
where = '\n\tand '.join([x for x in where if x])
print(where)

ifnull(is_deleted,0)
	and '2021-06-03' <= if (updated_at is null, created_at, updated_at)
	and if (updated_at is null, created_at, updated_at) < '2021-06-03 17:32:02'
	and (1)
